
# Calibración de la Cámara y Guardado de Parámetros con Visualización Ajustada

In [1]:
import numpy as np
import cv2 as cv
import glob

# Definir el tamaño del tablero de ajedrez (por ejemplo, 6x6 esquinas internas)
CHECKERBOARD = (6,6)

# Criterios de terminación
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Preparar puntos objeto
objp = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1,2)

# Arrays para almacenar puntos objeto y puntos imagen de todas las imágenes
objpoints = [] # puntos 3d en el espacio real (del mundo)
imgpoints = [] # puntos 2d en el plano de la imagen

# Obtener lista de imágenes
images = glob.glob('*.jpg')

for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Encontrar esquinas del tablero
    ret, corners = cv.findChessboardCorners(gray, CHECKERBOARD, None)

    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        # Dibujar y mostrar las esquinas
        cv.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
        
        # Redimensionar la imagen para que se muestre en una ventana más pequeña
        resize_width = 640
        resize_height = int(img.shape[0] * (resize_width / img.shape[1]))
        resized_img = cv.resize(img, (resize_width, resize_height))

        cv.imshow('Esquinas del Tablero', resized_img)
        cv.waitKey(500)

cv.destroyAllWindows()

# Calibración
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Guardar los parámetros de calibración y puntos clave para evitar reprocesar las imágenes
np.savez('calibration_params.npz', mtx=mtx, dist=dist, rvecs=rvecs, tvecs=tvecs, objpoints=objpoints, imgpoints=imgpoints)

# Calcular el error de reproyección
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error

print("Error total de reproyección: {}".format(mean_error/len(objpoints)))


Error total de reproyección: 0.18229606217871813


# Uso de los Parámetros Guardados, Corrección de Imagen y Visualización

In [6]:
import cv2 as cv
import numpy as np

# Cargar los parámetros de calibración
with np.load('calibration_params.npz') as data:
    mtx, dist, rvecs, tvecs = [data[i] for i in ('mtx', 'dist', 'rvecs', 'tvecs')]
    objpoints, imgpoints = [data[i] for i in ('objpoints', 'imgpoints')]

# Leer la imagen
img = cv.imread('calibration_images/prueba2.jpg')
h, w = img.shape[:2]

# Obtener la matriz nueva de la cámara y la región de interés
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

# Inicializar el mapa de corrección
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)

# Remapear la imagen
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)

# Recortar la imagen según la región de interés
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]

# Guardar la imagen corregida
cv.imwrite('calibration_images/calibresult.png', dst)

# Mostrar la imagen original y la imagen de-distorsionada en ventanas más pequeñas
cv.namedWindow('Imagen Original', cv.WINDOW_NORMAL)
cv.resizeWindow('Imagen Original', 600, 400)  # Cambia el tamaño a 600x400 píxeles
cv.imshow('Imagen Original', img)

cv.namedWindow('Imagen De-Distorsionada', cv.WINDOW_NORMAL)
cv.resizeWindow('Imagen De-Distorsionada', 800, 500)  # Cambia el tamaño a 600x400 píxeles
cv.imshow('Imagen De-Distorsionada', dst)

cv.waitKey(0)
cv.destroyAllWindows()

# Calcular el error de proyección
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2) / len(imgpoints2)
    mean_error += error

print("Error total: {}".format(mean_error / len(objpoints)))


Error total: 0.18229606217871813


# creacion del aruco y guardado en un archivo

In [3]:
import cv2
import numpy as np

# Parámetros de los marcadores ArUco
aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_6X6_50)
marker_id = 0  # ID del marcador que deseas generar
marker_size = 700  # Tamaño del marcador en píxeles

# Generar el marcador ArUco
marker_img = np.zeros((marker_size, marker_size), dtype=np.uint8)
marker_img = cv2.aruco.drawMarker(aruco_dict, marker_id, marker_size, marker_img, 1)

# Guardar el marcador como imagen
cv2.imwrite('Aruco/aruco_marker4.png', marker_img)

# Mostrar el marcador en una ventana
cv2.imshow('ArUco Marker', marker_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Implementacion de detector de intrusos en un área especifica usando aruco y detector de personas

In [5]:
import cv2
import numpy as np
import cv2.aruco as aruco
import tensorflow as tf
import tensorflow_hub as hub
import os
import winsound

# Factor de escala para aumentar el tamaño de la ventana
scale_factor = 3  # Aumenta este valor para hacer la ventana más grande

# Cargar el modelo MobileNet SSD v2 pre-entrenado de TensorFlow Hub
MODEL_PATH = "ssd_mobilenet_v2_model" # Asegúrate de que esta ruta sea correcta
detector = tf.saved_model.load(MODEL_PATH)

# Cargar los parámetros de la cámara desde el archivo de calibración
calibration_data = np.load('calibration_params.npz')
camera_matrix = calibration_data['mtx']
dist_coeffs = calibration_data['dist']

# Configuración inicial de ArUco
aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_50)  # ArUco de 6x6
aruco_params = aruco.DetectorParameters_create()

# Inicializar la cámara
cap = cv2.VideoCapture(0)   # 0 para la cámara web predeterminada

# Factor de desplazamiento para ampliar el área alrededor del marcador
expand_by = 100

# Variable para controlar cada cuántos cuadros procesar
process_every_n_frames = 5
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # Solo procesar 1 de cada process_every_n_frames cuadros
    if frame_count % process_every_n_frames != 0:
        continue

    # Corregir la distorsión de la imagen
    h, w = frame.shape[:2]
    new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(camera_matrix, dist_coeffs, (w, h), 1, (w, h))
    undistorted_frame = cv2.undistort(frame, camera_matrix, dist_coeffs, None, new_camera_matrix)

    # Recortar el área útil
    x, y, w, h = roi
    undistorted_frame = undistorted_frame[y:y+h, x:x+w]

    # Detección de marcadores ArUco en la imagen corregida
    corners, ids, _ = aruco.detectMarkers(undistorted_frame, aruco_dict, parameters=aruco_params)
    
    prohibited_area = None
    
    if ids is not None:
        for i in range(len(ids)):
            c = corners[i][0]
            
            # Expande cada esquina del área prohibida alrededor del marcador
            top_left = [c[0][0] - expand_by, c[0][1] - expand_by]
            top_right = [c[1][0] + expand_by, c[1][1] - expand_by]
            bottom_right = [c[2][0] + expand_by, c[2][1] + expand_by]
            bottom_left = [c[3][0] - expand_by, c[3][1] + expand_by]
            
            # Crear el área prohibida ampliada alrededor del marcador en el suelo
            prohibited_area = np.array([top_left, top_right, bottom_right, bottom_left], dtype=np.int32)
            
            # Dibujar el área prohibida en la imagen
            cv2.polylines(undistorted_frame, [prohibited_area], isClosed=True, color=(0, 255, 0), thickness=3)

    # Preprocesar la imagen para el modelo de detección de personas
    input_tensor = tf.convert_to_tensor(undistorted_frame)
    input_tensor = input_tensor[tf.newaxis, ...]

    # Realizar la detección
    detections = detector(input_tensor)

    # Procesar las detecciones
    boxes = detections['detection_boxes'][0].numpy()
    classes = detections['detection_classes'][0].numpy().astype(np.int32)
    scores = detections['detection_scores'][0].numpy()

    # Dibujar las detecciones en el frame
    height, width, _ = undistorted_frame.shape
    intrusion_detected = False
    
    for i in range(len(boxes)):
        if scores[i] > 0.5 and classes[i] == 1:  # class 1 es 'person'
            ymin, xmin, ymax, xmax = boxes[i]
            xmin = int(xmin * width)
            xmax = int(xmax * width)
            ymin = int(ymin * height)
            ymax = int(ymax * height)

            # Dibujar la caja de detección de persona
            cv2.rectangle(undistorted_frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            cv2.putText(undistorted_frame, f'Person: {scores[i]:.2f}', (xmin, ymin - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # Verificar si la persona está dentro del área prohibida
            if prohibited_area is not None:
                for (px, py) in [(xmin, ymin), (xmax, ymin), (xmax, ymax), (xmin, ymax)]:
                    if cv2.pointPolygonTest(prohibited_area, (px, py), False) >= 0:
                        intrusion_detected = True
                        break

    # Si se detecta una intrusión, mostrar alerta y activar sonido
    if intrusion_detected:
        cv2.putText(undistorted_frame, 'INTRUSION DETECTED', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        winsound.Beep(1000, 500)

    # Redimensionar el frame para mostrar una ventana más grande
    display_frame = cv2.resize(undistorted_frame, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)

    # Mostrar el video en tiempo real con el área prohibida y detecciones de personas
    cv2.imshow('Intrusion Detection', display_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()